In [1]:
import os


In [2]:
%pwd


'f:\\projects\\end-to-end-mlops-chicken-diseases-classification-project\\end-to-end-mlops-chicken-diseases-classification-project\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'f:\\projects\\end-to-end-mlops-chicken-diseases-classification-project\\end-to-end-mlops-chicken-diseases-classification-project'

In [7]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    kaggle_dataset:str
    local_data_file:Path
    unzip_dir:Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            kaggle_dataset=config.kaggle_dataset,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config


    

In [14]:
import os
import kaggle
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [20]:
from kaggle.api.kaggle_api_extended import KaggleApi

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        os.environ['KAGGLE_USERNAME']=os.environ.get('KAGGLE_USERNAME')
        os.environ['KAGGLE_KEY']=os.environ.get('KAGGLE_KEY')

        if not os.path.exists(self.config.local_data_file):
            try:
                api=KaggleApi()
                api.authenticate()

                api.dataset_download_files(
                    self.config.kaggle_dataset,
                    path=self.config.root_dir,
                    unzip=True
                )

                logger.info(f"Dataset downloaded to {self.config.root_dir}")
            except Exception as e:
                logger.exception(f"An error occurred while downloading the dataset: {e}")

        else:
           logger.info(f"File already exists at {self.config.local_data_file}") 

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Dataset extracted to {unzip_path}")



        


    
    






In [21]:
# creating the pipeline:

try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-02-25 00:12:41,573:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-02-25 00:12:41,575:INFO:common:yaml file: params.yaml loaded successfully]
[2024-02-25 00:12:41,576:INFO:common:created directory at: artifacts]
[2024-02-25 00:12:41,577:INFO:common:created directory at: artifacts/data_ingestion]
[2024-02-25 00:21:54,959:INFO:2134624008:Dataset downloaded to artifacts/data_ingestion]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/data.zip'